### 决策树创建过程：
* 检查现在标签是否都是相同的
* 是否还有特征<br>
**以上两个只要一个不满足就不能再继续**
### 决策树执行过程：
* 先调用fit
* fit调用create_tree
* 若可以继续分割，则选择最佳特征

### 样本和特征和特征值对应的关系：
 * 样本与特征是：**多对多**
 * 特征与特征值是：**多对多**

# 注意在每个步骤中使用的是全量数据还是部分子集

In [1]:
import numpy as np

### 信息熵计算
* 参数：划分数据子集标签的numpy.array
* 实现思路：<br>
  **1**、字典存储标签及其总数<br>
  **2**、遍历字典，计算每个标签的pi及log(pi)值，求和得到熵

##### 实现的错误：
* 未注意标签是一个一维数组，所以np.shape[0]
* entropy 的计算公式必须乘以 **prob**,且在计算前必须初始化0.0

In [2]:
def calEntro(label_array):
    label_sum = label_array.shape[0]#获取标签的总数，用于占比的计算
    label_dict = {}#用于存储标签及其对应的个数
    for curr_label in label_array:
        if curr_label not in label_dict.keys(): 
            label_dict[curr_label] = 0#如果不包含必须初始化为0
        label_dict[curr_label] += 1 # +1
    entropy = 0.0
    #根据公式进行计算
    for label in list(label_dict.keys()):
        prob = label_dict[label]/float(label_sum)
        entropy -= prob*np.log2(prob)
    return entropy

### 划分子数据集
* 目的：
  * 返回当前第i个特征取值为value时，包含的标签列表sub_y，用于计算熵值
  * 返回经过除去该特征后剩余的特征子集，用于以该结点为根再分裂的特征集
* 参数：
  * X 全量数据集
  * label_array 标签
  * idx 当前特征的下标
  * value 当前特征的特征值

In [3]:
def split_data_subset(X,label_array,idx,value):
    sub_idx = []# 空数组用来存放数据子集的，特征值/标签下标
    featVal = X[:,idx]#取出下标为idx特征的特征值
    X = X[:,[i for i in range(X.shape[1]) if i != idx]]#取出除去下标为idx特征后的特征子集
    #通过遍历当前特征值列表，取出特征值为val的数据子集(包含除去idx特征的特征子集和标签)
    for i in range(len(featVal)):
        if featVal[i] == value:
            sub_idx.append(i)#添加该下标
    return X[sub_idx,:],label_array[sub_idx]

##### split_data_subset测试

In [8]:
X = np.array([[1, 2, 0, 1, 0],
         [0, 1, 1, 0, 1],
         [1, 0, 0, 0, 1],
         [2, 1, 1, 0, 1],
         [1, 1, 0, 1, 1]])
label_array = np.array(['yes','yes','no','no','no'])
idx = 0
value = 0
split_data_subset(X,label_array,idx,value)

(array([[1, 1, 0, 1]]), array(['yes'], 
       dtype='<U3'))

### 选取最优特征
* 目的：
  * 给定X,y 通过比较信息增益选择出当前子集中的最优特征
* 参数：
  * X-当前子集所有特征列表 y-当前子集所有标签列表
* 实现思路：
  * 取出当前子集的特征子集
  * 遍历当前特征子集
  * 计算当前特征值的经验熵
  * 返回特征子集中信息增益最大的特征

In [ ]:
实现过程中的错误：
* 未注意在每个特征值，划分子集时，占整个特征权重，即sub_y占所有标签列表的比例
* 参数y的数据结构选择，是np array 还是 list

In [4]:
def select_best_feature(X,y):
    old_entropy = calEntro(y)#当前子集经验熵初始化
    #取出当前子集中特征子集
    feature_idx_list = X.shape[1]#取出特征子集下标
    best_info_gain = 0.0 #最大信息增益
    best_feature_idx = -1
    for idx in range(feature_idx_list):
        feature_val_list = X[:,idx]#取出下标为idx的所有特征值
        new_entroy = 0.0
        #划分数据集，计算经验熵
        unquie_feature_val = set(feature_val_list)
        for feature in unquie_feature_val:
            sub_x,sub_y = split_data_subset(X,y,idx,feature)
            prob = float(len(sub_y))/len(y)
            curr_entropy =prob * calEntro(sub_y)#计算经验熵时，只要传当前子集的标签列表即可
            new_entroy += curr_entropy #每个特征都会有entropy，所以应当在第一层for循环中声明该变量
        info_gain = old_entropy - new_entroy
        if (info_gain > best_info_gain):
            best_info_gain = info_gain
            best_feature_idx = idx
    return best_feature_idx

##### 测试 最优特征选择

In [ ]:
featureIndex = tuple(['x'+str(i) for i in range(X.shape[1])])

### 生成决策树
* 目的：
  * 根据当前的子集，生成决策子树
* 参数
  * X-该特征值下数据集 y-全量的标签列表 feature_tuple特征集合(注意数据结构的改变)
* 实现思路：
  * 停止生长的条件校验：
     * 当前子集中的标签列表都是一个标签
     * 当前子集无特征子集
  * 满足可生长条件：
     * 选择最优特征生长
  * 递归调用该过程
  * 此处使用非递归实现即从特征列表删除当前最优特征，利用该特征值切分后的子集继续生长(此处不再是使用全量数据)**理解错误**

In [ ]:
问题：
* 当前

In [5]:
def bulid_tree(X,label_array,feature_val_tup):
    label_list = list(label_array)#取出全量标签列表
    #决策树生长条件校验
    if label_list.count(label_array[0]) == len(label_list):
        return label_array[0]#
    if len(feature_val_tup) == 0:
        return 
    #满足生长条件：选择最优特征并生长
    
    best_feature_idx = select_best_feature(X,label_array)
    feature_val_list = list(feature_val_tup)#转为list remove 该特征
    best_feature_val = feature_val_list[best_feature_idx]
    feature_val_list.remove(best_feature_val)
    feature_val_tup = tuple(feature_val_list)
    #以当前的最优特征为根节点的子树{'x0': {}} {'x1': {}} 
    #从结果分析，x0特征划分后，其
    my_tree = {best_feature_val:{}}
    feature_val_list = X[:,best_feature_idx]#选取出最优特征的特征值
    feature_val_unique = set(feature_val_list)#对该特征的特征值去重
    for feature in feature_val_unique:
        sub_x,sub_y = split_data_subset(X,label_array,best_feature_idx,feature)#划分数据集
        my_tree[best_feature_val][feature] = bulid_tree(sub_x,sub_y,feature_val_tup)#递归调用
    return my_tree

fit函数：
* 参数：
   * X——全量的训练数据 lable_array——标签列表
* 实现思路：
   * 检查数据是否是**numpy.ndarray**类型
   * 生成feature_tup
   * 调用构建树函数

In [6]:
def fit(X,label_array):
    if isinstance(X,np.ndarray) & isinstance(label_array,np.ndarray):
        pass
    else:
        try:
            X = np.array(X)
            label_array = np.array(label_array)
        except:
            raise Exception('numpy.ndarray required for X label_array')
    feature_tup = tuple(['x'+str(i) for i in range(X.shape[1])])
    dec_tree = bulid_tree(X,label_array,feature_tup)
    return dec_tree

##### 测试fit、树生长、选择最优特征

In [13]:
X = [[1, 2, 0, 1, 0],
     [0, 1, 1, 0, 1],
     [1, 0, 0, 0, 1],
     [2, 1, 1, 0, 1],
     [1, 1, 0, 1, 1]]
y = ['yes', 'yes', 'no', 'no', 'no']

dec_tree = fit(X,y)
isinstance(dec_tree,dict)
# {0: 'yes', 1: {'x1': {0: 'no', 1: 'no', 2: 'yes'}}, 2: 'no'}

True

### predict预测
* **参数**：
  * X - 样本
* **目的**：
  * 对输入的样本X分类
* **实现思路**：
  * 校验输入的X是否是numpy.array
  * **classify()**--找出子集的所有label—这是个内部函数，：
     * **参数**：
       * tree-决策树 sample-单个样本
     * **思路**:
       * 根据tree，取出特征--注意此处是**特征**
       * 取出特征的idx，取出样本中的特征值
       * 根据特征值，根据字典取出val
         * 若Val为字典，则递归调用classify
         * 若Val不为字典，则返回label
  * 遍历样本**串行分类**即for循环实现。


In [10]:
def predict(X,tree):
    #校验
    if isinstance(X,np.ndarray):
        pass
    else:
        try:
            X = np.array(X)
        except:
            raise Exception('X required for numpy.ndarray')
#     tree 存储格式：字典的嵌套——递归 这里仅用三个结点
#     tree = {child_tree_1,child_tree_2,...,child_tree_n}
#     child_tree_N = {feature:{feature_val_1:label_1,feature_val_2:label_2,...,feature_val_n:label_n}}
#     child_tree_N  假设叶子结点为倒数第一层结点，即倒数的第二层结点
#     child_tree_N = {feature:{child_tree}}非叶子结点
#     leaf_node = n*{feature_val:lable}

# {'x0': {0: 'yes', 1: {'x1': {0: 'no', 1: 'no', 2: 'yes'}}, 2: 'no'}}
    def classify(tree,sample):
        feature = tree.keys()[0]#取出当前子树的特征
        child_tree_all = tree[feature]#取出当前子树
        feature_val = sample[int(feature[1:])]#从该样本中取出当前特征的特征值，feature[1:] 是取出特征idx
        child_tree_curr = child_tree_all[feature_val]---------------------
        if isinstance(tree[feature_val],dict):
            label = classify(child_tree_curr,sample)
        else:
            label = child_tree_curr
        return lable
    if len(X.shape) == 1:
        classify(tree,X)
    else:
        result = []
#         %debug
        for i in range(X.shape[0]):#遍历样本
            result.append(classify(X,tree))
        return result

In [78]:
X

[[1, 2, 0, 1, 0],
 [0, 1, 1, 0, 1],
 [1, 0, 0, 0, 1],
 [2, 1, 1, 0, 1],
 [1, 1, 0, 1, 1]]

In [14]:
dec_tree

{'x0': {0: 'yes', 1: {'x1': {0: 'no', 1: 'no', 2: 'yes'}}, 2: 'no'}}

In [15]:
predict(X,dec_tree)

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [ ]:
{'x0': {0: 'yes'}} 解释：是以x0 为根节点的，0 是特征值，yes是label 

### 信息熵的计算：

**1**、字典存储标签及其总数<br>
**2**、遍历字典，计算每个标签的pi及log(pi)值，求和得到熵

### 根据特征划分数据集：

In [ ]:
**1**、Xi有(去重后)N个特征值，在N个特征值下又可以划分为N类
**2**、

In [39]:
bestFeatStr = 'q'
myTree = {bestFeatStr:{}}
myTree

{'q': {}}

In [40]:
myTree[bestFeatStr][bestFeatStr] = 2
myTree

{'q': {'q': 2}}

In [42]:
bestFeatStr = 'q'
myTree[bestFeatStr][bestFeatStr] = 2

In [53]:
myTree= {'x0': {0: 'yes', 1: {'x1': {0: 'no', 1: 'no', 2: 'yes'}}, 2: 'no'}}

In [54]:
myTree['x0']

{0: 'yes', 1: {'x1': {0: 'no', 1: 'no', 2: 'yes'}}, 2: 'no'}

In [55]:
type(myTree)

dict

In [56]:
myTree['x0'][1]

{'x1': {0: 'no', 1: 'no', 2: 'yes'}}